In [11]:
import cv2
import numpy as np

import matplotlib.pyplot as plt

In [12]:
def create_gaussian_kernel():
    return np.array([1, 4, 6, 4, 1]) / 16.0

def apply_gaussian_filter(image):
    kernel = create_gaussian_kernel()
    if len(image.shape) == 3:
        filtered = np.zeros_like(image)
        for i in range(3):
            filtered[:,:,i] = cv2.sepFilter2D(image[:,:,i], -1, kernel, kernel)
        return filtered
    return cv2.sepFilter2D(image, -1, kernel, kernel)

def generate_gaussian_pyramid(image, levels):
    pyramid = [image]
    current = image.copy()
    for _ in range(levels):
        filtered = apply_gaussian_filter(current)
        current = filtered[::2, ::2]
        pyramid.append(current)
    return pyramid

def generate_laplacian_pyramid(image, levels):
    gaussian_pyramid = generate_gaussian_pyramid(image, levels)
    pyramid = []
    
    for i in range(levels):
        current = gaussian_pyramid[i]
        next_level = gaussian_pyramid[i + 1]
        expanded = cv2.resize(next_level, (current.shape[1], current.shape[0]))
        laplacian = current - expanded
        pyramid.append(laplacian)
    
    pyramid.append(gaussian_pyramid[-1])
    return pyramid

def create_region_mask(background_shape, region, feather=50):
    """Create a mask with stronger feathering for smoother blending"""
    mask = np.zeros(background_shape[:2])
    x, y, w, h = region
    
    # Create base mask
    mask[y:y+h, x:x+w] = 1
    
    # Apply stronger feathering
    mask = cv2.GaussianBlur(mask, (feather*2+1, feather*2+1), feather/3)
    return mask

def blend_images(background, foreground, mask, region, levels=6):
    """Blend with more pyramid levels for smoother transitions"""
    x, y, w, h = region
    foreground_resized = cv2.resize(foreground, (w, h))
    
    # Create canvas for foreground
    canvas = background.copy()
    canvas[y:y+h, x:x+w] = foreground_resized
    
    # Generate pyramids
    mask_pyramid = generate_gaussian_pyramid(mask, levels)
    background_pyramid = generate_laplacian_pyramid(background, levels)
    canvas_pyramid = generate_laplacian_pyramid(canvas, levels)
    
    # Blend pyramids
    blended_pyramid = []
    for i in range(len(background_pyramid)):
        mask_level = np.expand_dims(mask_pyramid[i], axis=2) if len(background_pyramid[i].shape) == 3 else mask_pyramid[i]
        blended = mask_level * canvas_pyramid[i] + (1 - mask_level) * background_pyramid[i]
        blended_pyramid.append(blended)
    
    return np.clip(reconstruct_from_pyramid(blended_pyramid), 0, 1)

def reconstruct_from_pyramid(pyramid):
    current = pyramid[-1]
    for level in reversed(pyramid[:-1]):
        expanded = cv2.resize(current, (level.shape[1], level.shape[0]))
        current = expanded + level
    return current

def create_composition():
    # Read and normalize images
    background = cv2.imread('photos/P200.jpg')
    dog1 = cv2.imread('photos/dog1.jpg')
    dog2 = cv2.imread('photos/dog2.jpg')
    cat = cv2.imread('photos/cat.jpg')
    bench = cv2.imread('photos/bench.jpg')
    nick = cv2.imread('photos/nick.png')
    
    if background is None or dog1 is None or dog2 is None or cat is None or bench is None or nick is None:
        raise ValueError("Failed to load one or more images")
    
    # Convert to RGB and normalize
    background = cv2.cvtColor(background, cv2.COLOR_BGR2RGB).astype(float) / 255
    dog1 = cv2.cvtColor(dog1, cv2.COLOR_BGR2RGB).astype(float) / 255
    dog2 = cv2.cvtColor(dog2, cv2.COLOR_BGR2RGB).astype(float) / 255
    cat = cv2.cvtColor(cat, cv2.COLOR_BGR2RGB).astype(float) / 255
    bench = cv2.cvtColor(bench, cv2.COLOR_BGR2RGB).astype(float) / 255
    nick = cv2.cvtColor(nick, cv2.COLOR_BGR2RGB).astype(float) / 255
    
    # Define better positioned regions (x, y, width, height)
    regions = [
        (100, 200, 400, 300),    # dog1 - larger and lower
        (600, 150, 400, 300),    # dog2 - right side
        (300, 400, 350, 350),    # cat - center bottom
        (50, 500, 500, 200),     # bench - bottom left
        (800, 300, 300, 300)     # nick - right side middle
    ]
    
    # Start with background
    result = background.copy()
    
    # Blend each image sequentially with improved parameters
    images = [dog1, dog2, cat, bench, nick]
    for img, region in zip(images, regions):
        # Create mask with stronger feathering
        mask = create_region_mask(result.shape, region, feather=60)
        
        # Blend with more pyramid levels
        result = blend_images(result, img, mask, region, levels=6)
    
    return result

In [13]:
try:
    # Create and display composition
    result = create_composition()
        
    plt.figure(figsize=(15, 10))
    plt.imshow(result)
    plt.axis('off')
    plt.title('Final Composition')
    plt.show()

    # Save the result
    plt.imsave('improved_composition.png', result)
        
except Exception as e:
    print(f"Error creating composition: {e}")


Error creating composition: Failed to load one or more images
